In [68]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re, time

pd.options.display.max_colwidth = 1000

In [69]:
sys.path.append(os.path.abspath(".."))
from loadFile import getFileList, getDirList, fileToDataFrame

# ディレクトリ 内のメールファイルを読み込む
directory_path = "/Users/taroaso/myprojects/OpenIE/wiki-research-l/2020-July"
file_list = getFileList(directory_path)

# テキストファイルをデータフレームに格納する
mail_df, thread_df = fileToDataFrame(file_list)

In [111]:
s_list = []
bullet_list = []
for i,content in mail_df['Content'].items():
    cleaned_lines = []
    original_lines = content.splitlines()
    for line in original_lines:
        if line == '':
            continue 
        elif re.match(r'\W{2,}.+',line):
            bullet_list.append(line)
        elif re.match(r'_{2,}.+',line):
            bullet_list.append(line)
        elif re.match(r'^(On \w{3}, \w{3} [0-9]{1,2}, [0-9]{4} at [0-9]{1,2}:[0-9]{1,2} (A|P)M .+)',line) is None:
            cleaned_lines.append(line)
        else:
            break
    cleaned_message = ' '.join(map(str, cleaned_lines))
    s_list.append(cleaned_message)
mail_df['Cleaned_Content'] = s_list

In [113]:
len(bullet_list)

984

In [109]:
mail_df[['Content','Cleaned_Content']]

,Content,Cleaned_Content
0,"Paul,\nSorry, I did not read your response carefully.\nWill try the wikimapper API as suggested.\n\nThanks a lot!\n\nOn Tue, Jul 21, 2020 at 1:05 PM Paul Albert <mrpaulalbert at gmail.com> wrote:\n\n>\n> I've been using the Wikimapper API mentioned here:\n>\n>\n> https://opendata.stackexchange.com/questions/5467/mapping-all-wikipedia-urls-to-wikidata-entity-ids\n>\n>\n> Regards,\n>\n> Paul Albert\n>\n> ﻿On 7/21/20, 12:39 PM, ""Wiki-research-l on behalf of Haifeng Zhang"" <\n> wiki-research-l-bounces at lists.wikimedia.org on behalf of\n> haifengz.suny at gmail.com> wrote:\n>\n> Hi all,\n> Are you aware of any API that maps wikidata QIDs to their article Page\n> IDs?\n>\n> Thanks,\n> Haifeng\n> _______________________________________________\n> Wiki-research-l mailing list\n> Wiki-research-l at lists.wikimedia.org\n> https://lists.wikimedia.org/mailman/listinfo/wiki-research-l\n> _______________________________________________\n> Wiki-research-l mailing...","Paul, Sorry, I did not read your response carefully. Will try the wikimapper API as suggested. Thanks a lot!"
1,"Hi all,\n\nThe next Research Showcase will be live-streamed on Wednesday, July 15, at\n9:30 AM PDT/16:30 UTC.\n\nWikipedia is one of the most important online resources for health\ninformation. This has been especially highlighted during the Covid-19\npandemic: since the beginning of the year more than 5000 articles related\nto Covid-19 have been created receiving more than 400M pageviews.\nTherefore, for this month’s showcase our two invited speakers will help us\nget a better understanding of the state of medical knowledge in Wikipedia.\nIn the first talk, Denise Smith will give an overview on how Wikipedia's\nhealth content is used by different audiences (public, students, or\npractitioners). In the second talk, Giovanni Colavizza will present results\non how editors on Wikipedia find, select, and integrate scientific\ninformation on Covid-19 into Wikipedia articles.\n\nYouTube stream: https://www.youtube.com/watch?v=qIV26lWrD9c\n\nAs usual, you can join the conversation on IRC ...","Hi all, The next Research Showcase will be live-streamed on Wednesday, July 15, at 9:30 AM PDT/16:30 UTC. Wikipedia is one of the most important online resources for health information. This has been especially highlighted during the Covid-19 pandemic: since the beginning of the year more than 5000 articles related to Covid-19 have been created receiving more than 400M pageviews. Therefore, for this month’s showcase our two invited speakers will help us get a better understanding of the state of medical knowledge in Wikipedia. In the first talk, Denise Smith will give an overview on how Wikipedia's health content is used by different audiences (public, students, or practitioners). In the second talk, Giovanni Colavizza will present results on how editors on Wikipedia find, select, and integrate scientific information on Covid-19 into Wikipedia articles. YouTube stream: https://www.youtube.com/watch?v=qIV26lWrD9c As usual, you can join the conversation on IRC at #wikimedia-research...."
2,"Hi Stella,\n\nThanks for clarifying. So unfortunately in this research, we are only\nlooking for researchers and developers who are familiar with ORES.\n\nBest,\nEthan\n\nOn Tue, Jun 30, 2020 at 3:17 PM Stella Yu <stellayu at gmail.com> wrote:\n\n> Hi Ethan,\n>\n> Thank you for your reply. I do not have any familiarity with ORES. In terms\n> of Wikipedia, I am a Wikipedia user. More specifically, I am paid\n> contributor to Wikipedia. This means I am a Conflict of Interest (COI)\n> editor. Is this helpful?\n>\n> Warmly,\n>\n> Stella\n>\n>\n>\n> On Tue, Jun 30, 2020 at 12:03 PM Ethan Ye <ziningy1 at andrew.cmu.edu> wrote:\n>\n> > Hi Stella,\n> >\n> > Thanks for your interest! Before we start scheduling, we would like to\n> know\n> > a bit more about you.\n> > Are you a researcher/developer in Wikipedia or if you have any experience\n> > working with ORES?\n> >\n> > Than

In [ ]:
'''
from preProcess import cleaningText
s_list = []
for i,content in mail_df['Content'].items():
    sentences = cleaningText(content)
    s_list.append(sentences)
mail_df['Cleaned_Content'] = s_list
'''

In [107]:
#　センテンスのリストを作る
sentence_list = []
# 文章全体に対する前処理
for content in mail_df['Cleaned_Content']:
    content = re.sub(r'\n{1,}',' ',content)
    content = re.sub(r'\s{2,}',' ',content)
    content = re.sub(r'_{2,}',' ',content)
    sentences = tokenize.sent_tokenize(content)
    # 文に対する前処理
    for i,sentence in enumerate(sentences):
        sentence = re.sub(r'(>|){2,}|(< ){2,}',' ',sentence)
        #if len(sentence.split()) <= 5:
         #   continue
#        elif len(sentence.split()) >= 100:
#            continue
        #else:
        sentence_list.append(sentence)
sentence_list

ata we need?',
 ' Warmly,  Mackenzie Lemieux  --  Mackenzie Lemieux  mackenzie.lemieux at gmail.com  cell: 416-806-0041  220 Gilmour Avenue  Toronto, Ontario  M6P 3B4     Wiki-research-l mailing list  Wiki-research-l at lists.wikimedia.org  https://lists.wikimedia.org/mailman/listinfo/wiki-research-l     Wiki-research-l mailing list  Wiki-research-l at lists.wikimedia.org  https://lists.wikimedia.org/mailman/listinfo/wiki-research-l  ------------------------------  Message: 2  Date: Fri, 10 Jul 2020 17:25:34 +0200 (CEST)  From: serge.autexier at dfki.de (Serge Autexier)  To: wiki-research-l at lists.wikimedia.org  Subject: [Wiki-research-l] CICM 2020, July 26-31: Call for Online  Participation  Message-ID: <20200710152534.D461C335C8F5 at gigondas.localdomain Content-Type: text/plain; charset="UTF-8"  =====================================================================  CALL FOR PARTICIPATION  CICM 2020 -- Conference on Intelligent Computer Mathematics  https://cicm-conference.org/2020

In [90]:
sent_length = {}
for i, sentence in enumerate(sentence_list):
    sent_length[i] = len(sentence.split())

In [91]:
len(sentence_list)

212

In [92]:
id_length = sorted(sent_length.items(), key=lambda x:x[1], reverse=True)
sent_cnt_list = []
for tpl in id_length:
    sent_cnt_list.append([tpl[0],tpl[1],sentence_list[tpl[0]]])

In [93]:
sent_cnt_df = pd.DataFrame(sent_cnt_list, columns=('idx','cnt','sentence'))
sent_cnt_df.to_csv('/Users/taroaso/myprojects/OpenIE/analysis/output/sent_cnt_df2.csv')